In [16]:
import requests
from bs4 import BeautifulSoup
import json
import re
from concurrent.futures import ThreadPoolExecutor
import os
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

workers = os.cpu_count()

headers = {
    "authority": "www.panvel.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "app-token": "ZYkPuDaVJEiD",
    "cookie": "_gcl_au=1.1.1785829145.1706019376; _pm_id=105001706019380552; _fbp=fb.1.1706019380895.894699130; __ssds=2; __ssuzjsr2=a9be0cd8e; __uzmaj2=b56d0445-77c6-457c-8f32-ce6688aca01b; __uzmbj2=1706019382; LojaVirtualPanvelUserId=e11013f7-201d-43b7-aca7-bf072521406b; _tt_enable_cookie=1; _ttp=_PbkVaYYq3wrHCo-t-4JgO_Rd3B; sessionId=e35c6bca-fb4c-4423-b015-84e4fb04d044; _gid=GA1.2.2048000362.1708637826; voxusmediamanager_registered_first_page=true; voxusmediamanager_id=17011060649700.17260056520497491by7up3dmmr; voxusmediamanager__ip=179.222.150.231; _pm_sid=107761708651218313; search=api; shoppingCartId=65d7f3ae3460767753de1951; _dc_gtm_UA-5035710-1=1; appName=home; _gat_UA-78262398-1=1; _uetsid=891f9080d1ca11eeb380d14ad03e81fd; _uetvid=fb281a00b9f911ee9a9463db05681391; _ga_YPSVJXM42F=GS1.2.1708651170.5.1.1708651730.60.0.0; __uzmcj2=695657619062; __uzmdj2=1708651730; cto_bundle=VaToIl80NGtFQjdFYVU5REhNU3BaRWdYQndWdmsyVzAlMkJLZjc3TzJ6bzZCbVd4YXJyd0R0eTVQUEFkdVJSTHlMZHRPVDZhWDNGY2Z3YW5VR0NyUlR0Y1hOTUQ2Y0lLQ1VHeUZkQUFwRWQ2ZU14UVdqR3BuZEdnQjVEWWFVb0gwMEF3TWhaWGg0RGxTT3JyTDhCOFRMQzAlMkI3aUd2T2JrWjQwRm5OWE5yVVo3U1NqVWxVQmN2d1VzWEMwbVBDVmNuM2xraXM4bjdMVmFEWUNGMEJPWXptY2JUJTJCJTJCZkElM0QlM0Q; _dd_s=rum=2&id=c197027c-7f5f-40d7-85af-42a268e373f7&created=1708651163929&expire=1708652654820; _ga=GA1.1.830842076.1706019377; _ga_WYT638QTZR=GS1.1.1708651168.7.1.1708651755.33.0.0; az_asm=hDxLHMG8K2ZY4fMIYGKBIGsbhf27pMli6p197nhXFUnRVGXjIaptCVG1NpZkcJ+Jxu/qd3JSLJGo4Pnl; az_botm=b55a533c5b08490e6325c603e080c933",
    "finger-print": "0c8d9302d12f31a5c0d6b1e415e39769",
    "referer": "https://www.panvel.com/",
    "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sessionid": "e35c6bca-fb4c-4423-b015-84e4fb04d044",
    "shoppingcartid": "65d7f3ae3460767753de1951",
    "source": "Desktop",
    "theme": "LIGHT",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "x-kl-kfa-ajax-request": "Ajax_Request"
}

### Extraindo as categorias e seu número de paginas

In [3]:
categorias = []

url = 'https://www.panvel.com/api/v1/category/menu'
response = requests.get(url, headers=headers)
if response.status_code == 200:
    data = response.json()
    categories = data.get("categories", [])
    for category in categories:
        cat = category.get("description")
        id = category.get("id")
        href = category.get("link")
        categoria = {'Categoria': cat, 'ID': id, 'Href': href}
        categorias.append(categoria)

else:
    print(f"Error in request. Status code: {response.status_code}")

df_categorias = pd.DataFrame(categorias)

In [25]:
def pegar_ultima_pagina(soup):
    try:
        pagination_list = soup.find('ul', class_='pagination-list')
        last_page_button = pagination_list.find_all('button')[-2]
        last_page_number = int(last_page_button.get_text(strip=True))
        return last_page_number
    except Exception as e:
        print(f"Erro ao obter a última página: {e}")
        return None

driver = webdriver.Chrome()

pags_categorias = []

for href in df_categorias['Href']:
    url = f'https://www.panvel.com{href}'
    driver.get(url)

    # Aguarde até que a página carregue
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.pagination-list')))

    # Criar soup a partir do HTML da página
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Obter o número da última página
    ultima_pagina = pegar_ultima_pagina(soup)

    # Adicionar informações da categoria ao DataFrame
    pags_cat = {'Href': href, 'UltimaPagina': ultima_pagina}
    pags_categorias.append(pags_cat)

# Fechar o navegador
driver.quit()
df_paginas = pd.DataFrame(pags_categorias)
df_categorias = pd.merge(df_categorias, df_paginas, on='Href')
df_categorias.to_csv('df_categorias.csv', index=False)


### Pegar a ID dos produtos para acessar a API de cada um deles. Fiz separadamente pois acabava perdendo a conexão ao iterar por todas as categorias

In [33]:
row_dermocosmeticos = df_categorias.loc[df_categorias['Categoria'] == 'Dermocosméticos']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_dermocosmeticos = []

# Iterar sobre as categorias
for index, row in row_dermocosmeticos.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_dermocosmeticos.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_dermocosmeticos = pd.DataFrame(ids_dermocosmeticos, columns=['ids'])

In [38]:
row_perfumes = df_categorias.loc[df_categorias['Categoria'] == 'Perfumes']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_perfumes = []

# Iterar sobre as categorias
for index, row in row_perfumes.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_perfumes.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_perfumes = pd.DataFrame(ids_perfumes, columns=['ids'])

In [41]:
row_medicamentos = df_categorias.loc[df_categorias['Categoria'] == 'Medicamentos']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_medicamentos = []

# Iterar sobre as categorias
for index, row in row_medicamentos.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_medicamentos.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_medicamentos = pd.DataFrame(ids_medicamentos, columns=['ids'])

In [42]:
row_panvel = df_categorias.loc[df_categorias['Categoria'] == 'Produtos Panvel']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_panvel = []

# Iterar sobre as categorias
for index, row in row_panvel.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_panvel.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_panvel = pd.DataFrame(ids_panvel, columns=['ids'])

In [45]:
row_infantil = df_categorias.loc[df_categorias['Categoria'] == 'Infantil E Gestante']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_infantil = []

# Iterar sobre as categorias
for index, row in row_infantil.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_infantil.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_infantil = pd.DataFrame(ids_infantil, columns=['ids'])

In [46]:
row_beleza = df_categorias.loc[df_categorias['Categoria'] == 'Beleza']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_beleza = []

# Iterar sobre as categorias
for index, row in row_beleza.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_beleza.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_beleza = pd.DataFrame(ids_beleza, columns=['ids'])

In [49]:
row_diario = df_categorias.loc[df_categorias['Categoria'] == 'Cuidado Diário']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_diario = []

# Iterar sobre as categorias
for index, row in row_diario.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_diario.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_diario = pd.DataFrame(ids_diario, columns=['ids'])

In [53]:
row_bemestar = df_categorias.loc[df_categorias['Categoria'] == 'Bem Estar']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_bemestar = []

# Iterar sobre as categorias
for index, row in row_bemestar.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_bemestar.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_bemestar = pd.DataFrame(ids_bemestar, columns=['ids'])

In [54]:
row_clinic = df_categorias.loc[df_categorias['Categoria'] == 'Panvel Clinic']

def extrair_ids(url):
    match = re.search(r'/p-(\d+)', url)
    if match:
        return match.group(1)
    else:
        return None

driver = webdriver.Chrome()
ids_clinic = []

# Iterar sobre as categorias
for index, row in row_clinic.iterrows():
    url = f'https://www.panvel.com{row["Href"]}'
    ultima_pagina = row['UltimaPagina']

    # Iterar sobre as páginas da categoria
    for page in range(1, ultima_pagina + 1):
        current_url = f'{url}?currentPage={page}'
        driver.get(current_url)

        try:
            # Aguarde até que os elementos sejam visíveis
            wait = WebDriverWait(driver, 10)
            product_links = wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://www.panvel.com/panvel/"][href*="/p-"]'))
            )
        except TimeoutException:
            # Se não houver mais produtos, pare o loop
            if "Não existem mais produtos" in driver.page_source:
                break
            else:
                continue

        # Extrair IDs dos produtos
        links_produto = [link.get_attribute('href') for link in product_links]
        ids_produto = [extrair_ids(url_produto) for url_produto in links_produto if extrair_ids(url_produto)]
        ids_clinic.extend(ids_produto)

# Fechar o navegador
driver.quit()
df_ids_clinic = pd.DataFrame(ids_clinic, columns=['ids'])

In [57]:
dfs = [df_ids_beleza, df_ids_bemestar, df_ids_clinic, df_ids_dermocosmeticos, df_ids_diario, df_ids_infantil, df_ids_medicamentos, df_ids_panvel, df_ids_perfumes]
df_ids = pd.concat(dfs, ignore_index=True)
df_ids = df_ids.drop_duplicates().reset_index(drop=True)
df_ids.to_csv('df_ids.csv')

### Extraindo as informações

In [70]:
def extrair_infos(id):
    try:
        api_url = f'https://www.panvel.com/api/v2/catalog/{id}'
        response = requests.get(api_url, headers=headers)
        if response.status_code == 200:
            product_data = response.json()  # Mudança aqui

            # Verificar se product_data é um dicionário
            if isinstance(product_data, dict):  # Mudança aqui
                nome = product_data.get('name', '')
                ean = product_data.get('ean', '')
                marca = product_data.get('brandName', '')
                descricao = product_data.get('description', '')
                preco_s = product_data.get('originalPrice', '')
                preco_c = product_data.get('discount', {}).get('dealPrice', '')
                porcentagem = product_data.get('discount', {}).get('discountPercentage', '')

                # Construir o dicionário com as informações e adicionar à lista
                info = {
                    'Produto': nome,
                    'EAN': ean,
                    'Marca': marca,
                    'Descrição': descricao,
                    'Preço sem desconto': preco_s,
                    'Preço com desconto': preco_c,
                    '% de desconto': f'{porcentagem}%',
                    'Farmácia': 'Preço Popular',
                    'Região': 'Sudeste',
                    'Cidade': 'São Paulo'
                }
                pbar.update(1)

                return [info]  # Mudança aqui

            else:
                print(f"{id} não retornou um dicionário de produto.")
                return None

        print(f"Nenhum produto encontrado para {id}")
        return None
    except Exception as e:
        print(f"Erro ao processar {id}: {str(e)}")
        return None
    
infos = []

# Inicializar o tqdm
with tqdm(total=len(df_ids)) as pbar:
    with ThreadPoolExecutor(max_workers=workers) as executor:
        results = executor.map(extrair_infos, df_ids['ids'])
        infos = [info for info in results if info is not None]

# Converter a lista de dicionários em um DataFrame do pandas
df_infos = pd.DataFrame([item[0] for item in infos])
df_infos.to_csv('df_infos.csv', index=False)

  0%|          | 0/24814 [00:00<?, ?it/s]

100%|██████████| 24814/24814 [22:41<00:00, 18.22it/s] 
